In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np
import re


In [20]:
class WebScraper:
    def __init__(self, url):
        self.url = url

    def scraper(self):
        res = requests.get(self.url)
        soup = BeautifulSoup(res.text, 'lxml')

        data = {
            'Title': [], 
            'Description': [],
            'Links': []
        }

        head = soup.find_all('div', class_='clearfix text-formatted field field--name-field-text-content field--type-text-long field--label-hidden field__item')

        def clean_text(text):
            cleanr = re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', text)
            return ' '.join(cleantext.split())

        for i in head:
            # Extract the titles
            u_t = i.select('ul')
            if u_t:
                first = u_t[0]
                li_tags = first.find_all('li')

                for part in li_tags:
                    parts = part.get_text()
                    data["Title"].append(parts)   #append in data in "Title" column
            
            # Extract data under each title
            des = i.select('ul')
            for k, j in enumerate(des):
                if k == 0 or k == 2:
                    continue
                desc = j.get_text()
                clean_desc = clean_text(desc)
                data["Description"].append(clean_desc.strip())  #append in data in "Description" column

            link = i.find_all('p')
            count = 0  # Initialize a counter

            for k in link:
                if count == 0:
                    count += 1
                    continue  # Skip the first <p> tag

                if count >= len(link) - 1:
                    break  # Stop when we reach the second last <p> tag

                a_tags = k.find_all('a')
                for a in a_tags:
                    if 'anchors' in a.get('class', []):
                        continue  # Skip <a> tags with class 'anchors'

                    data["Links"].append(a.get('href'))

                count += 1  # Increment the counter after processing each <p> tag

            second_last_link = i.find_all('ul')
            if second_last_link:
                s_l_l = second_last_link[-2]
                li = s_l_l.find_all('li')
                
                for l in li:
                    a_tag = l.find('a')
                    data["Links"].append(a_tag.get('href'))
                    
            last_link = i.find_all('p')
            if last_link:
                last_p = last_link[-1]
                a_tags = last_p.find_all('a')
                for a in a_tags:
                    if 'anchors' in a.get('class', []):
                        continue  # Skip <a> tags with class 'anchors'
                    data["Links"].append(a.get('href'))

        df = pd.DataFrame.from_dict(data)
        df.to_csv("data.csv", index=False)
        print("CSV created successfully!!")

        # return df

# Example usage:
scraper = WebScraper("https://www.earthdata.nasa.gov/engage/open-data-services-and-software/api")
scraper.scraper()


CSV created successfully!!
